## 1. 전세사기_all.csv 내용 크롤링

In [31]:
#내용 크롤링
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import time
import re
df=pd.read_csv('./전세사기_all.csv')
df_res=[]
for idx in df.index:
    try:
        html=requests.get(df['link'][idx])
        soup=bs(html.text, 'lxml')
        tags=soup.find('div','question-content')
        try:
            content=tags.find('div','c-heading__content').text.replace('\t','').strip('\n') # 질의
        except:
            content=None

        try:
            date=soup.find_all('span','c-userinfo__info')[0].text.replace('작성일','') #조회수
        except:
            date=None

        try:
            views=int(soup.find_all('span','c-userinfo__info')[1].text.replace('조회수 ','').replace(',','')) #조회수
        except:
            views=None
            
        try:
            tag=tags.find('a','tag-list__item tag-list__item--category').text.replace('태그 디렉터리Ξ','').strip() #질의 태그
        except:
            tag=None
            
        try:
            content_a=soup.find('div', '_endContentsText c-heading-answer__content-user').text #응답
        except:
            content=None
            
        tmp={
            'No':idx,
            '내용':content,
            '날짜':date,
            '조회수':views,
            '태그':tag,
            '내용1(응답)':content_a
        }
        df_res.append(tmp)
    except:
        pass
    if idx%1000==0:
        tmp=pd.DataFrame(df_res)
        tmp.to_csv(f'./data_contents/{idx}.csv')
        df_res=[]
        
last_df=pd.DataFrame(df_res)
last_df.to_csv('./data_contents/last_df.csv')

In [32]:
len(df)

2252

# 2. 크롤링한 데이터 취합(contents_df)

In [65]:
from glob import glob
fileList=glob('./data_contents/*.csv')
#데이터 취합
contents_df=pd.concat([pd.read_csv(i,index_col=0) for i in fileList],ignore_index=True )

In [66]:
contents_df

,No,내용,날짜,조회수,태그,내용1(응답)
0,0,재개발 말이나오는 지역에 갭투자를 했는데요 재작년 말쯤이었고 원래는 세대원으로 살다...,2022.10.05,48.0,"분양, 청약",재개발의 경우는 서울 투자한 곳에 실거주하지 않아도 조합원 자격이 ...
1,1,부동산 경매와 갭투자는 어떤구조로 되어있는지 각각 알려주세요 수익금은 둘중 뭐가더 ...,2022.09.04,370.0,경매,"안녕하세요, 질문자님​부동산경매와 갭투자에 대해 질문을 주셨네요!​..."
2,2,NaN,2022.09.13,90.0,매매,질문에 답변드리겠습니다. 갭투자를 한 후 현 임차인이 퇴거를 하는 ...
3,3,제가 김포쪽으로 신축빌라를 하나 사서 제 지인에게 전세를 놓고 싶은데 전세가 안되는...,2022.10.02,62.0,매매,네전세는 다 놓을수있어요근데 요즘 집값이 떨어지는 추세라 전셋값도 ...
4,4,개인회생 개시가 나서 월변제금을 납부하고있고 아직 인가결정은 안났습니다.그런데 아버...,2022.06.05,216.0,"신용, 파산",개인회생 개시가 나서 월변제금을 납부하고있고 아직 인가결정은 안났습...
...,...,...,...,...,...,...
2247,2247,보험이나 대출홍보 글 모두 신고하였습니다질문 보고 답글 다시길...손해보험 전세대출...,2012.09.06,749.0,금융상품담보대출,보험에 대한 종류도 정말 많거니와 혼자서 알아보기란 쉽지않기 때문입니다 원래 보험은...
2248,2248,처음 자취해서 챙겨야 할것들이랑고시텔 계약할때 계약서 쓰나요 주의사항은요?월세는 어...,2016.10.26,120.0,이사,고시텔 고시원을 처음 구하시는 것 같네요일반 자취방보다 훨씬 간단하고 쉽기 때문에크...
2249,2249,원룸을 전세로 저가 구입한다음 다른사람에게 월세 받으면서 팔아도되는건가요!!??내공...,2016.06.23,278.0,부동산,전세는 구입하는 게 아니라 2년동안 월세 없이몫돈을 내고 사는 것 입니다.세입자에게...
2250,2250,현재 전월세 계약을 맺고 전세거주중입니다. 민간임대 아파트에 당첨되어 계약을 진행하...,2022.07.08,25.0,임대차,본인이 거주하지 않더라도 본인 명의로 전월세 계약을 맺는것은 이중계...


In [67]:
#null 확인
contents_df.isnull().sum()

No           0
내용         250
날짜         171
조회수        171
태그         171
내용1(응답)      0
dtype: int64

In [68]:
# fillna: 문자열은 ''으로 숫자열은 0르로 채움
for i in contents_df:
    if contents_df[i].dtype=='O':
        contents_df[i]=contents_df[i].fillna('')
    else:
        contents_df[i]=contents_df[i].fillna(0)


In [69]:
#null 확인
contents_df.isnull().sum()

No         0
내용         0
날짜         0
조회수        0
태그         0
내용1(응답)    0
dtype: int64

In [75]:
contents_df

,No,내용,날짜,조회수,태그,내용1(응답)
0,0,재개발 말이나오는 지역에 갭투자를 했는데요 재작년 말쯤이었고 원래는 세대원으로 살다...,2022.10.05,48.0,"분양, 청약",재개발의 경우는 서울 투자한 곳에 실거주하지 않아도 조합원 자격이 ...
1,1,부동산 경매와 갭투자는 어떤구조로 되어있는지 각각 알려주세요 수익금은 둘중 뭐가더 ...,2022.09.04,370.0,경매,"안녕하세요, 질문자님​부동산경매와 갭투자에 대해 질문을 주셨네요!​..."
2,2,,2022.09.13,90.0,매매,질문에 답변드리겠습니다. 갭투자를 한 후 현 임차인이 퇴거를 하는 ...
3,3,제가 김포쪽으로 신축빌라를 하나 사서 제 지인에게 전세를 놓고 싶은데 전세가 안되는...,2022.10.02,62.0,매매,네전세는 다 놓을수있어요근데 요즘 집값이 떨어지는 추세라 전셋값도 ...
4,4,개인회생 개시가 나서 월변제금을 납부하고있고 아직 인가결정은 안났습니다.그런데 아버...,2022.06.05,216.0,"신용, 파산",개인회생 개시가 나서 월변제금을 납부하고있고 아직 인가결정은 안났습...
...,...,...,...,...,...,...
2247,2247,보험이나 대출홍보 글 모두 신고하였습니다질문 보고 답글 다시길...손해보험 전세대출...,2012.09.06,749.0,금융상품담보대출,보험에 대한 종류도 정말 많거니와 혼자서 알아보기란 쉽지않기 때문입니다 원래 보험은...
2248,2248,처음 자취해서 챙겨야 할것들이랑고시텔 계약할때 계약서 쓰나요 주의사항은요?월세는 어...,2016.10.26,120.0,이사,고시텔 고시원을 처음 구하시는 것 같네요일반 자취방보다 훨씬 간단하고 쉽기 때문에크...
2249,2249,원룸을 전세로 저가 구입한다음 다른사람에게 월세 받으면서 팔아도되는건가요!!??내공...,2016.06.23,278.0,부동산,전세는 구입하는 게 아니라 2년동안 월세 없이몫돈을 내고 사는 것 입니다.세입자에게...
2250,2250,현재 전월세 계약을 맺고 전세거주중입니다. 민간임대 아파트에 당첨되어 계약을 진행하...,2022.07.08,25.0,임대차,본인이 거주하지 않더라도 본인 명의로 전월세 계약을 맺는것은 이중계...


# 3. 전세사기_all.csv 와 contents_df 합치기(열방향)

In [78]:
df

,Unnamed: 0,title,link,description
0,0,서울 갭투자,https://kin.naver.com/qna/detail.naver?d1id=4&...,재개발 말이나오는 지역에 <b>갭투자</b>를 했는데요 재작년 말쯤이었고 원래는 세...
1,1,부동산 경매 갭투자,https://kin.naver.com/qna/detail.naver?d1id=4&...,부동산 경매와 <b>갭투자</b>는 어떤구조로 되어있는지 각각 알려주세요 수익금은....
2,2,갭투자에 관한 질문이에요.,https://kin.naver.com/qna/detail.naver?d1id=4&...,<b>갭투자</b>에 관한 질문이에요. 집값이 떨어질 때 제가 집을 매매하려고 하잖...
3,3,김포 빌라 갭투자,https://kin.naver.com/qna/detail.naver?d1id=4&...,... 아무 집이든 <b>갭투자</b>가 가능한건가요? 안녕하세요... 집과사람들...
4,4,개인회생 개시결정후 갭투자,https://kin.naver.com/qna/detail.naver?d1id=6&...,... 그런데 아버지가 부동산업을 하시는데 부동산을 <b>갭투자</b>를 전세 1억...
...,...,...,...,...
2247,50910,손해보험사 전세대출에 관해 질문입니다,https://kin.naver.com/qna/detail.naver?d1id=4&...,... 혹시 <b>이중계약</b>이라고 해야하나;; 이런식으로 대출 받아 다른 집으...
2248,50911,제가 고시텔에 들어가려고 하는데요,https://kin.naver.com/qna/detail.naver?d1id=8&...,... 요즘 학생들을 대상으로 부동산 사기꾼들 많습니다... 경매넘어간 물건을 <b...
2249,50912,원룸 전세 해서 원룸 팔기,https://kin.naver.com/qna/detail.naver?d1id=4&...,... 내공처먹으로 온놈 ㅋㅋ신고한다 그런 경우를 <b>전월세</b>라고 하는데 ...
2250,50947,이중 전월세계약 - 처벌 및 과태료 부과여부,https://kin.naver.com/qna/detail.naver?d1id=4&...,... 본인이 거주하지 않더라도 본인 명의로 <b>전월세</b> 계약을 맺는것은 <...


In [79]:
contents_df

,No,내용,날짜,조회수,태그,내용1(응답)
0,0,재개발 말이나오는 지역에 갭투자를 했는데요 재작년 말쯤이었고 원래는 세대원으로 살다...,2022.10.05,48.0,"분양, 청약",재개발의 경우는 서울 투자한 곳에 실거주하지 않아도 조합원 자격이 ...
1,1,부동산 경매와 갭투자는 어떤구조로 되어있는지 각각 알려주세요 수익금은 둘중 뭐가더 ...,2022.09.04,370.0,경매,"안녕하세요, 질문자님​부동산경매와 갭투자에 대해 질문을 주셨네요!​..."
2,2,,2022.09.13,90.0,매매,질문에 답변드리겠습니다. 갭투자를 한 후 현 임차인이 퇴거를 하는 ...
3,3,제가 김포쪽으로 신축빌라를 하나 사서 제 지인에게 전세를 놓고 싶은데 전세가 안되는...,2022.10.02,62.0,매매,네전세는 다 놓을수있어요근데 요즘 집값이 떨어지는 추세라 전셋값도 ...
4,4,개인회생 개시가 나서 월변제금을 납부하고있고 아직 인가결정은 안났습니다.그런데 아버...,2022.06.05,216.0,"신용, 파산",개인회생 개시가 나서 월변제금을 납부하고있고 아직 인가결정은 안났습...
...,...,...,...,...,...,...
2247,2247,보험이나 대출홍보 글 모두 신고하였습니다질문 보고 답글 다시길...손해보험 전세대출...,2012.09.06,749.0,금융상품담보대출,보험에 대한 종류도 정말 많거니와 혼자서 알아보기란 쉽지않기 때문입니다 원래 보험은...
2248,2248,처음 자취해서 챙겨야 할것들이랑고시텔 계약할때 계약서 쓰나요 주의사항은요?월세는 어...,2016.10.26,120.0,이사,고시텔 고시원을 처음 구하시는 것 같네요일반 자취방보다 훨씬 간단하고 쉽기 때문에크...
2249,2249,원룸을 전세로 저가 구입한다음 다른사람에게 월세 받으면서 팔아도되는건가요!!??내공...,2016.06.23,278.0,부동산,전세는 구입하는 게 아니라 2년동안 월세 없이몫돈을 내고 사는 것 입니다.세입자에게...
2250,2250,현재 전월세 계약을 맺고 전세거주중입니다. 민간임대 아파트에 당첨되어 계약을 진행하...,2022.07.08,25.0,임대차,본인이 거주하지 않더라도 본인 명의로 전월세 계약을 맺는것은 이중계...


In [80]:
df_res=pd.concat([df,contents_df],axis=1)[['title','내용','내용1(응답)','description','태그','날짜','조회수','link']]

In [81]:
df_res

,title,내용,내용1(응답),description,태그,날짜,조회수,link
0,서울 갭투자,재개발 말이나오는 지역에 갭투자를 했는데요 재작년 말쯤이었고 원래는 세대원으로 살다...,재개발의 경우는 서울 투자한 곳에 실거주하지 않아도 조합원 자격이 ...,재개발 말이나오는 지역에 <b>갭투자</b>를 했는데요 재작년 말쯤이었고 원래는 세...,"분양, 청약",2022.10.05,48.0,https://kin.naver.com/qna/detail.naver?d1id=4&...
1,부동산 경매 갭투자,부동산 경매와 갭투자는 어떤구조로 되어있는지 각각 알려주세요 수익금은 둘중 뭐가더 ...,"안녕하세요, 질문자님​부동산경매와 갭투자에 대해 질문을 주셨네요!​...",부동산 경매와 <b>갭투자</b>는 어떤구조로 되어있는지 각각 알려주세요 수익금은....,경매,2022.09.04,370.0,https://kin.naver.com/qna/detail.naver?d1id=4&...
2,갭투자에 관한 질문이에요.,,질문에 답변드리겠습니다. 갭투자를 한 후 현 임차인이 퇴거를 하는 ...,<b>갭투자</b>에 관한 질문이에요. 집값이 떨어질 때 제가 집을 매매하려고 하잖...,매매,2022.09.13,90.0,https://kin.naver.com/qna/detail.naver?d1id=4&...
3,김포 빌라 갭투자,제가 김포쪽으로 신축빌라를 하나 사서 제 지인에게 전세를 놓고 싶은데 전세가 안되는...,네전세는 다 놓을수있어요근데 요즘 집값이 떨어지는 추세라 전셋값도 ...,... 아무 집이든 <b>갭투자</b>가 가능한건가요? 안녕하세요... 집과사람들...,매매,2022.10.02,62.0,https://kin.naver.com/qna/detail.naver?d1id=4&...
4,개인회생 개시결정후 갭투자,개인회생 개시가 나서 월변제금을 납부하고있고 아직 인가결정은 안났습니다.그런데 아버...,개인회생 개시가 나서 월변제금을 납부하고있고 아직 인가결정은 안났습...,... 그런데 아버지가 부동산업을 하시는데 부동산을 <b>갭투자</b>를 전세 1억...,"신용, 파산",2022.06.05,216.0,https://kin.naver.com/qna/detail.naver?d1id=6&...
...,...,...,...,...,...,...,...,...
2247,손해보험사 전세대출에 관해 질문입니다,보험이나 대출홍보 글 모두 신고하였습니다질문 보고 답글 다시길...손해보험 전세대출...,보험에 대한 종류도 정말 많거니와 혼자서 알아보기란 쉽지않기 때문입니다 원래 보험은...,... 혹시 <b>이중계약</b>이라고 해야하나;; 이런식으로 대출 받아 다른 집으...,금융상품담보대출,2012.09.06,749.0,https://kin.naver.com/qna/detail.naver?d1id=4&...
2248,제가 고시텔에 들어가려고 하는데요,처음 자취해서 챙겨야 할것들이랑고시텔 계약할때 계약서 쓰나요 주의사항은요?월세는 어...,고시텔 고시원을 처음 구하시는 것 같네요일반 자취방보다 훨씬 간단하고 쉽기 때문에크...,... 요즘 학생들을 대상으로 부동산 사기꾼들 많습니다... 경매넘어간 물건을 <b...,이사,2016.10.26,120.0,https://kin.naver.com/qna/detail.naver?d1id=8&...
2249,원룸 전세 해서 원룸 팔기,원룸을 전세로 저가 구입한다음 다른사람에게 월세 받으면서 팔아도되는건가요!!??내공...,전세는 구입하는 게 아니라 2년동안 월세 없이몫돈을 내고 사는 것 입니다.세입자에게...,... 내공처먹으로 온놈 ㅋㅋ신고한다 그런 경우를 <b>전월세</b>라고 하는데 ...,부동산,2016.06.23,278.0,https://kin.naver.com/qna/detail.naver?d1id=4&...
2250,이중 전월세계약 - 처벌 및 과태료 부과여부,현재 전월세 계약을 맺고 전세거주중입니다. 민간임대 아파트에 당첨되어 계약을 진행하...,본인이 거주하지 않더라도 본인 명의로 전월세 계약을 맺는것은 이중계...,... 본인이 거주하지 않더라도 본인 명의로 <b>전월세</b> 계약을 맺는것은 <...,임대차,2022.07.08,25.0,https://kin.naver.com/qna/detail.naver?d1id=4&...


# 4. 저장

In [82]:
df_res.to_csv('./최종_전세사기_all_.csv')